In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import plotly.express as px
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False    # 负号显示的解决方案


import toml
from pprint import pprint


from dash import Dash, Input, Output
# from grpc import server
import plotly.express as px
import plotly.graph_objs as go
import dash
from dash import dcc                  # 交互式组件
from dash import html                 # 代码转html
from dash.dependencies import Input, Output         # 回调
from jupyter_dash import JupyterDash
import json
# pip3 install jupyter-server-proxy
# pip3 install jupyter-dash

### __读取数据__

In [2]:
data = pd.read_excel('./2022年售后维修在线编辑汇总(1).xlsx')
# data = pd.read_excel(r'D:\jupyterproject\comtom数据\新建文件夹\2022年售后维修在线编辑汇总(2).xlsx')

In [3]:
data['问题现象'].isnull().sum()

583

In [4]:
print(data.shape)
data['问题现象'].isnull().count()

(4526, 17)


4526

### __填补缺失值__

In [5]:
data['问题现象'].fillna('mask1', inplace=True)
print(data['问题现象'].isna().sum())

0


In [6]:
# data.dropna(axis=0, subset=['问题原因'], inplace=True)
data['问题原因'].fillna('mask2', inplace=True)
print(data['问题原因'].isna().sum())

0


In [7]:
# data.dropna(axis=0, subset=['解决方案'], inplace=True)
data['解决方案'].fillna('mask3', inplace=True)
print(data['解决方案'].isna().sum())

0


### __读取配置文件__

In [8]:
# 需要统一的文字描述
Unity_config = toml.load('./UnifiedWriting.toml')
# pprint(Unity_config['unity']['problemappearance'])
# pprint(Unity_config['unity']['problemcause'])
# pprint(Unity_config['unity']['solution'])

# 需要添加 "更新"的配件名
addword_config = toml.load('./AddReplaceWord.toml')
# pprint(addword_config)
needchange_parts = addword_config['needadd_parts']['parts']

### __处理文本信息__

In [9]:
# 考虑将部分列的英文标点转换成中文标点，并去掉末尾的换行符号

# 规格型号、问题现象、问题原因、解决方案这几列需要转换
table = {ord(f):ord(t) for f,t in zip(
        # u'()!?;[],，',
        # u'（）！？；【】//'                 #这里顺便把，也换成了/
        u'(),',
        u'（），'
    )
}
def translate_E2C(col):
    for i in range(len(data)):
        if type(data.iloc[i, col]) == str:
            data.iloc[i, col] = data.iloc[i, col].strip('\n') #去掉换行
            data.iloc[i, col] = data.iloc[i, col].translate(table)


translate_E2C(4)
translate_E2C(8)
translate_E2C(9)
translate_E2C(10)

In [10]:
# 考虑将解决方案列中 UF/V 和 A/V 中的/删去

def convert_UFV():
    temp = data[data['解决方案'].str.contains('UF/') & 
                    data['解决方案'].str.contains('V')]['解决方案']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], 10]
        # print(list(a)[0])
        ind = list(a)[0].index('UF/')
        # print(ind)
        data.iloc[data.index==temp.index[i], 10] = list(a)[0][:ind+2] + list(a)[0][ind+3:] #用字符串切片的方式进行删除
        # print(data.iloc[data.index==temp.index[i], 10].tolist()[0])
def convert_AV():
    temp = data[data['解决方案'].str.contains('A/') & 
                    data['解决方案'].str.contains('V')]['解决方案']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], 10]
        # print(list(a)[0])
        ind = list(a)[0].index('A/')
        # print(ind)
        data.iloc[data.index==temp.index[i], 10] = list(a)[0][:ind+1] + list(a)[0][ind+2:] #用字符串切片的方式进行删除
        # print(data.iloc[data.index==temp.index[i], 10].tolist()[0])


convert_UFV()
convert_AV()
data[data['解决方案'].str.contains('UF/') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕
data[data['解决方案'].str.contains('A/') & 
        data['解决方案'].str.contains('V')]['解决方案'].value_counts() #验证转换完毕

Series([], Name: 解决方案, dtype: int64)

In [11]:
# 考虑将问题现象列的 1.2.转换为/形式，统一一下

def convert_123():
    temp = data[data['问题现象'].str.startswith('1.')]['问题现象']
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], 8]
        # print(list(a)[0])
        if list(a)[0][:2]=='1.': #这句 if判断没有也可以
            b = list(a)[0][2:].split('.') #利用 '.'对字符串进行分割
            for j in range(len(b)):
                if j < len(b)-1: #这一步是删除经过 split分割后的多余数字
                    b[j] = b[j][:-1]
            data.iloc[data.index==temp.index[i], 8] = '/'.join(b)
            # print(data.iloc[data.index==temp.index[i], 8].tolist()[0])
convert_123()


data[data['问题现象'].str.startswith('1.')]['问题现象'].value_counts() #验证转换完毕

Series([], Name: 问题现象, dtype: int64)

In [12]:
# 考虑部分意义相同的情况，需要进行修改，统一改成相同的表述
# 因为有些数据是形如：不上电/网络连接不上
#                   不上电-网络连接不上

def handle_backslash_123(data, temp, changedstr, judgestr='', col='问题现象'):
    '''
        data: 是一个 pandas的 DataFrame
        temp: data经过 .str.contains()筛选后的结果
        judgestr: 判断字符, 例如网络连接不上的判断字符就是 '网络', 有时候不存在 '/'以及 '1.2.3.'的情况也可以选择不传入
        changedstr: 修改后的字符, 例如网络连接不上的修改后的字符就是 '网络不通'
        i: temp的下标, 因为需要遍历来按照索引进行修改
    '''
    if col != '问题现象':
        loc = data.columns.tolist().index(col)
    else:
        loc = data.columns.tolist().index('问题现象') #找到问题现象出现的列索引
    for i in range(len(temp.index)):
        a = data.iloc[data.index==temp.index[i], loc]
        if '/' in list(a)[0] or '，' in list(a)[0]: #利用 '/'对字符串进行分割
            # print(list(a)[0])
            if '/' in list(a)[0]:
                split_str = '/'
            elif '，' in list(a)[0]:
                split_str = '，'
            b = list(a)[0].split(split_str)
            for j in range(len(b)):
                if judgestr in b[j]:
                    b[j] = changedstr
                    break
            data.iloc[data.index==temp.index[i], loc] = split_str.join(b)
        else:
            data.iloc[data.index==temp.index[i], loc] = changedstr

In [13]:
def Unity(col, temp=[]):
    cols = ['问题现象', '问题原因', '解决方案']
    Englishcol = ['problemappearance', 'problemcause', 'solution']
    cols_dict = dict(zip(cols, Englishcol))
    
    keys = list(Unity_config['unity'][cols_dict[col]]['or'].keys())
    for i in keys:
        print("-----", i, ": ")
        if Unity_config['unity'][cols_dict[col]]['or'][i]:
            screen = '|'.join(Unity_config['unity'][cols_dict[col]]['or'][i])
            # print(screen)
            temp = data[col].str.contains(screen)
        if Unity_config['unity'][cols_dict[col]]['and'][i]:
            for j in Unity_config['unity'][cols_dict[col]]['and'][i]:
                # print(j)
                # print(len(temp))
                if len(temp)!=0:
                    if j[0] == '~':
                        temp &= ~data[col].str.contains(j[1:])
                    else:
                        temp &= data[col].str.contains(j)
                else:
                    if j[0] == '~':
                        temp = ~data[col].str.contains(j[1:])
                    else:
                        temp = data[col].str.contains(j)
        temp = data[temp][col]
        # print(temp)
        handle_backslash_123(data, temp, changedstr=i,
                            judgestr=Unity_config['unity'][cols_dict[col]]['str'][i], col=col)
        temp = []

Unity(col='问题现象')
Unity(col='问题原因')
Unity(col='解决方案')

----- 不上电 : 
----- 4G不在线 : 
----- 无声音输出 : 
----- 网络不通 : 
----- 声音异常 : 
----- 搜索不到终端 : 
----- 不出串口 : 
----- 电源指示灯不亮 : 
----- 网络指示灯不亮 : 
----- 不断重启 : 
----- 检测无问题 : 
----- 开关电源坏 : 
----- 通讯模块坏 : 
----- 功放芯片坏 : 
----- 网络芯片坏 : 
----- SIM卡槽坏 : 
----- FB2电感坏 : 
----- 升级程序 : 
----- 更换网口座子RJ45 : 
----- 更换功放芯片MP7770 : 


In [14]:
# # 问题现象列
# keys = list(Unity_config['unity']['problemappearance']['or'].keys())

# for i in keys:
#     print("-----", i, ": ")
#     if Unity_config['unity']['problemappearance']['or'][i]: #先解决or
#         screen = '|'.join(Unity_config['unity']['problemappearance']['or'][i])
#         # print(screen)
#         temp = data['问题现象'].str.contains(screen)
#     if Unity_config['unity']['problemappearance']['and'][i]: #再解决and
#         for j in Unity_config['unity']['problemappearance']['and'][i]:
#             # print(j)
#             # print(len(temp))
#             if len(temp)!=0:
#                 if j[0] == '~':
#                     temp &= ~data['问题现象'].str.contains(j[1:])
#                 else:
#                     temp &= data['问题现象'].str.contains(j)
#             else:
#                 if j[0] == '~':
#                     temp = ~data['问题现象'].str.contains(j[1:])
#                 else:
#                     temp = data['问题现象'].str.contains(j)
#     temp = data[temp]['问题现象']
#     # print(temp)
#     handle_backslash_123(data, temp, changedstr=i,
#                         judgestr=Unity_config['unity']['problemappearance']['str'][i], col='问题现象')
#     temp = []

In [15]:
# # 问题原因列
# keys = list(Unity_config['unity']['problemcause']['or'].keys())

# for i in keys:
#     print("-----", i, ": ")
#     if Unity_config['unity']['problemcause']['or'][i]:
#         screen = '|'.join(Unity_config['unity']['problemcause']['or'][i])
#         # print(screen)
#         temp = data['问题原因'].str.contains(screen)
#     if Unity_config['unity']['problemcause']['and'][i]:
#         for j in Unity_config['unity']['problemcause']['and'][i]:
#             # print(j)
#             # print(len(temp))
#             if len(temp)!=0:
#                 if j[0] == '~':
#                     temp &= ~data['问题原因'].str.contains(j[1:])
#                 else:
#                     temp &= data['问题原因'].str.contains(j)
#             else:
#                 if j[0] == '~':
#                     temp = ~data['问题原因'].str.contains(j[1:])
#                 else:
#                     temp = data['问题原因'].str.contains(j)
#     temp = data[temp]['问题原因']
#     # print(temp)
#     handle_backslash_123(data, temp, changedstr=i,
#                         judgestr=Unity_config['unity']['problemcause']['str'][i], col='问题原因')
#     temp = []

In [16]:
# # 解决方案列
# keys = list(Unity_config['unity']['solution']['or'].keys())

# for i in keys:
#     print("-----", i, ": ")
#     if Unity_config['unity']['solution']['or'][i]:
#         screen = '|'.join(Unity_config['unity']['solution']['or'][i])
#         # print(screen)
#         temp = data['解决方案'].str.contains(screen)
#     if Unity_config['unity']['solution']['and'][i]:
#         for j in Unity_config['unity']['solution']['and'][i]:
#             # print(j)
#             # print(len(temp))
#             if len(temp)!=0:
#                 if j[0] == '~':
#                     temp &= ~data['解决方案'].str.contains(j[1:])
#                 else:
#                     temp &= data['解决方案'].str.contains(j)
#             else:
#                 if j[0] == '~':
#                     temp = ~data['解决方案'].str.contains(j[1:])
#                 else:
#                     temp = data['解决方案'].str.contains(j)
#     temp = data[temp]['解决方案']
#     # print(temp)
#     handle_backslash_123(data, temp, changedstr=i,
#                         judgestr=Unity_config['unity']['solution']['str'][i], col='解决方案')
#     temp = []

In [17]:
temp = data[(data['问题原因'].str.contains('但是不行'))]['问题原因'].value_counts()
temp

红灯慢闪，开关电源坏，但是不行，更换电源    3
Name: 问题原因, dtype: int64

In [20]:
# 处理多问题现象的情况

def split_multiple(problem, cnt):
    '''
        主要用于分离含有 '/'或者 '，'的问题现象
        注意要先忽略掉 '/升级程序', 再分离
    '''
    i = 0
    while i < len(problem):
        # 先处理含有多情况下的升级程序情况
        if '/升级程序' in problem[i] or "升级程序/" in problem[i]:
            if '/升级程序' in problem[i]:
                problem[i] = ''.join(problem[i].split('/升级程序'))
            if '升级程序/' in problem[i]:
                problem[i] = ''.join(problem[i].split('升级程序/'))
        if '，升级程序' in problem[i] or "升级程序，" in problem[i]:
            if '，升级程序' in problem[i]:
                problem[i] = ''.join(problem[i].split('，升级程序'))
            if '升级程序，' in problem[i]:
                problem[i] = ''.join(problem[i].split('升级程序，'))
        # 再处理正常情况
        if '/' in problem[i] or '，' in problem[i]:
            if '/' in problem[i]:
                temp = problem[i].split('/')
            elif '，' in problem[i]:
                temp = problem[i].split('，')
            problem += temp
            cnt += [cnt[i]]*len(temp)
            problem.pop(i)
            cnt.pop(i)
        else:
            i += 1

def sum_multiple(problem, cnt):
    '''
        有时候存在一个规格型号有多个问题现象, 那可能需要和之前单一的情况合并起来考虑
        problem: 是一个列表, 存放着需要绘制图像的文本数据
        cnt: 是一个列表, 存放着需要绘制图像的数值数据
        先根据文本长度排序, 短的文本在前面, 防止丢失不一样的解决方案
        有时候存在空字符串 '', 需要删掉
        之后去重
        注意在 needchange_parts中的配件需要添加更换二字
    '''
    temp = sorted(enumerate(problem), key=lambda x:len(x[1]), reverse=False)
    idx = [i[0] for i in temp]
    problem = [i[1] for i in temp]
    cnt = [cnt[i] for i in idx]

    # # 去掉 ''的情况
    # if '' in problem:
    #     ind = problem.index('')
    #     problem.pop(ind)
    #     cnt.pop(ind)

    # 去重
    pro_dict = {}
    for i in range(len(problem)):
        if problem[i] != '': #忽略 ''的情况
            if problem[i] in needchange_parts:
                problem[i] = '更换'+problem[i]
            flag = 0
            # # 对于含有相同文本的情况，只取最长的文本
            # for key in pro_dict.keys(): #这一步是在去重
            #     if problem[i] in key: #短的文本在长的里面
            #         pro_dict[key] += cnt[i] #长的值增加
            #         flag = 1
            #         break
            if flag == 0:
                if problem[i] not in pro_dict:
                    pro_dict[problem[i]] = cnt[i]
                else:
                    pro_dict[problem[i]] += cnt[i]
    
    # print(list(pro_dict.keys()), '\n', list(pro_dict.values()))
    # problem, cnt = list(pro_dict.keys()), list(pro_dict.values())
    return list(pro_dict.keys()), list(pro_dict.values()) #字典解包

In [21]:
# 记录 '成品' 与 '板件' 的型号
product_type = list(data['产品类型'].value_counts().index)[:-1]

model = {} #字典用于存放成品或者板件的型号
for i in product_type:
    model[i] = list(data[data['产品类型']==i]['规格型号'].value_counts().index)

In [27]:
# options、value、children？？？
app = JupyterDash('Chained Callbacks')
all_options = model

app.layout = html.Div([
    html.Div(
        [html.H4('售后服务数据')]
    ),
    html.Div(
        [dcc.Graph(id = "graph")],
        # style = dict(width = '64%', display = 'inline-block')
    ),
    html.Div(
        [html.P("产品类型:"),
        dcc.Dropdown(id = 'types',
                    options = [{'label': k, 'value': k} for k in all_options.keys()],
                    value = '成品', #初始值为成品
                    clearable = False
        ),
        html.P("规格型号:"),
        dcc.Dropdown(id = 'specifications',
                    clearable = False
        ),
        html.P("列名:"),
        dcc.Dropdown(id = 'columns',
                    options = ['问题现象', '问题原因', '解决方案'],
                    value = '问题现象', #初始值为问题现象
                    clearable = False
        )],
        style = dict(width = '40%', display = 'inline-block') #设置下拉列表的大小
    )
])

@app.callback(
    Output('specifications', 'options'), #options即下面的return是一个列表嵌套一个字典
    [Input('types', 'value')])
def set_type_options(select_type): #select_type是传入的参数，表示选择成品还是组件
    return [{'label': i, 'value': i} for i in all_options[select_type]]

@app.callback(
    Output('specifications', 'value'),
    [Input('specifications', 'options')])
def set_specification_value(available_options): #available_options表示上面返回的列表嵌套字典
    return available_options[0]['value'] #取列表的第一项，找到字典键为value对应的值，这里选择键label也一样的


@app.callback(
    Output("graph", "figure"), #需要更新的输出，即图像
    [Input("types", "value"), #需要更新的输入，有三个：产品类型、规格型号、列名
    Input("specifications", "value"),
    Input("columns", "value"),])
def generate_chart(types, specifications, columns):
    texts = []
    values = []
    i = len(texts)
    temp = data[data['规格型号']==specifications][columns].value_counts()

    texts.append(list(temp.index))
    values.append(list(temp))

    # 是否忽略缺失值
    def check_exist(textdata):
        ind = []
        check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
        for str in check_str:
            if str in textdata:
                ind.append(textdata.index(str))
        return ind
    ind = sorted(check_exist(texts[i]), reverse=True)
    for j in ind:
        texts[i].pop(j)
        values[i].pop(j)

    split_multiple(texts[i], values[i])
    texts[i], values[i] = sum_multiple(texts[i], values[i])

    labels, numsdata = texts[i], values[i]
    df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
    fig = px.pie(
                df,
                values = '次数',
                names = columns,
                hole = 0.3,
                title = '型号 '+ specifications +' 的主要'+ columns,
    )
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',)
    return fig


if __name__ == '__main__':
    # app.run_server(mode='inline', width='80%', height=800)
    app.run_server(mode='external', host='127.0.0.1', port=8060, debug=True)

Dash app running on http://127.0.0.1:8060/


In [63]:
# ipynb转py
# !jupyter nbconvert --to python 数据处理.ipynb

In [ ]:
# 端口被占用时
# !netstat -ano|findstr 8050
# !taskkill/F /pid ...

### 尝试划分子图

In [229]:
# options、value、children？？？
from base64 import decode


app = JupyterDash('Chained Callbacks')
all_options = model

app.layout = html.Div([
    html.Div(
        [html.H4('售后服务数据')]
    ),
    html.Div(
        [html.P("产品类型:"),
        dcc.Dropdown(id = 'types',
                    options = [{'label': k, 'value': k} for k in all_options.keys()],
                    value = '成品', #初始值为成品
                    clearable = False
        ),
        html.P("规格型号:"),
        dcc.Dropdown(id = 'specifications',
                    clearable = False
        ),
        html.P("列名:"),
        dcc.Dropdown(id = 'columns',
                    options = ['问题现象', '问题原因', '解决方案'],
                    value = '问题现象', #初始值为问题现象
                    clearable = False
        )],
        style = dict(width = '30%', display = 'inline-block', float = 'right', padding = '80px 0px 0px 0px') #设置下拉列表的大小
                                                                                                                # padding是上左下右
    ),
    html.Div([
            dcc.Graph(id = "graph"),
        ],
        style = dict(width = '70%', display = 'inline-block',)
    ),
    html.Div([
            dcc.Graph(id = "该现象的问题原因"), 
            html.Div(id = "output")
        ],
        style = dict(width = '50%', display = 'inline-block',)
    ), 
    html.Div([
            dcc.Graph(id = "该现象的解决方案"),
        ],
        style = dict(width = '50%', display = 'inline-block', float='right')
    ),    
])

# 选择产品类型
@app.callback(
    Output('specifications', 'options'), #options即下面的return是一个列表嵌套一个字典
    [Input('types', 'value')])
def set_type_options(select_type): #select_type是传入的参数，表示选择成品还是组件
    return [{'label': i, 'value': i} for i in all_options[select_type]]

# 选择规格型号
@app.callback(
    Output('specifications', 'value'),
    [Input('specifications', 'options')])
def set_specification_value(available_options): #available_options表示上面返回的列表嵌套字典
    return available_options[0]['value'] #取列表的第一项，找到字典键为value对应的值，这里选择键label也一样的

# 根据问题现象来画图
@app.callback(
    Output("graph", "figure"), #需要更新的输出，即图像
    [Input("types", "value"), #需要更新的输入，有三个：产品类型、规格型号、列名
    Input("specifications", "value"),
    Input("columns", "value"),])
def generate_chart(types, specifications, columns):
    texts = []
    values = []
    i = len(texts)
    temp = data[data['规格型号']==specifications][columns].value_counts()

    texts.append(list(temp.index))
    values.append(list(temp))

    # 是否忽略缺失值
    def check_exist(textdata):
        ind = []
        check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
        for str in check_str:
            if str in textdata:
                ind.append(textdata.index(str))
        return ind
    ind = sorted(check_exist(texts[i]), reverse=True)
    for j in ind:
        texts[i].pop(j)
        values[i].pop(j)

    split_multiple(texts[i], values[i])
    texts[i], values[i] = sum_multiple(texts[i], values[i])

    labels, numsdata = texts[i], values[i]
    df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
    

    ind = df.sort_values(by='次数', inplace=False, ascending=False)['次数'].index.tolist()
    # print(ind)
    colors = ['lightslategray',] * len(df)
    if len(ind) >= 10:
        length = 3
    else:
        length = 1
    for i in range(length):
        colors[ind[i]] = 'crimson'
    fig = go.Figure(
        data=[
            go.Bar(
                x = df['次数'],
                y = df[columns],
                text = df['次数'],
                marker_color = colors,
                orientation = 'h', #横向的条形图
            )
        ]
    )
    fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                        title_text='型号 '+ specifications +' 的主要'+ columns,)
    return fig

# 可视化某一问题现象对应的问题原因
@app.callback(
    Output("该现象的问题原因", "figure"), #需要更新的输出，即图像
    # Output("output", "children"), #可以查看输出结果
    [Input("graph", "hoverData"),
    Input("specifications", "value"),
    Input("columns", "value")])
def synchronization_1(hoverdata, specifications, columns):
    a = str(hoverdata)
    a = a[60:].split(",")[2].split(":")[1].split("'")[1]
    if columns == '问题现象':
        # return a[60:].split(",")[2].split(":")[1].split("'")[1]
        temp = data[data['规格型号']==specifications]
        temp = temp[temp['问题现象']==a]['问题原因'].value_counts()
        texts = list(temp.index)
        values = list(temp)
        dff = pd.DataFrame(zip(texts, values), columns=['问题原因', '次数'])
        fig = go.Figure(
            data=[
                go.Bar(
                    x = dff['次数'],
                    y = dff['问题原因'],
                    text = dff['次数'],
                    orientation = 'h', #横向的条形图
                )
            ]
        )
        fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
        fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                            title_text='型号 '+ specifications +' 的主要问题原因')
        return fig

# 可视化某一问题现象对应的解决方案
@app.callback(
    Output("该现象的解决方案", "figure"), #需要更新的输出，即图像
    [Input("graph", "hoverData"),
    Input("specifications", "value"),
    Input("columns", "value")])
def synchronization_1(hoverdata, specifications, columns):
    a = str(hoverdata)
    a = a[60:].split(",")[2].split(":")[1].split("'")[1]
    if columns == '问题现象':
        # return a[60:].split(",")[2].split(":")[1].split("'")[1]
        temp = data[data['规格型号']==specifications]
        temp = temp[temp['问题现象']==a]['解决方案'].value_counts()
        texts = list(temp.index)
        values = list(temp)
        dff = pd.DataFrame(zip(texts, values), columns=['解决方案', '次数'])
        fig = go.Figure(
            data=[
                go.Bar(
                    x = dff['次数'],
                    y = dff['解决方案'],
                    text = dff['次数'],
                    orientation = 'h', #横向的条形图
                )
            ]
        )
        fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
        fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                            title_text='型号 '+ specifications +' 的主要解决方案')
        return fig


if __name__ == '__main__':
    # app.run_server(mode='inline', width='80%', height=800)
    app.run_server(mode='external', host='127.0.0.1', port=8060, debug=True)

Dash app running on http://127.0.0.1:8060/


In [228]:
# a = "{'points': [{'curveNumber': 0, 'pointNumber': 5, 'pointIndex': 5, 'x': 146, 'y': '4G不在线', 'label': '4G不在线', 'value': 146, 'text': 146, 'marker.color': 'crimson', 'bbox': {'x0': 1074.65, 'x1': 1074.65, 'y0': 289.18, 'y1': 308.82}}]}"
# # json.loads(a)
# b = "{'points': [{'curveNumber': 0, 'pointNumber': 6, 'pointIndex': 6, 'x': 6, 'y': '无声音输出', 'label': '无声音输出', 'value': 6, 'text': 6, 'marker.color': 'lightslategray', 'bbox': {'x0': 157.31, 'x1': 157.31, 'y0': 264.64, 'y1': 284.27}}]}"
# eval(a)['points'][0]['label']

# aa = a[60:].split(",")[2].split(":")[1].split("'")[1]
# aa

# bb = b[60:].split(",")[2].split(":")[1].split("'")[1]
# bb

In [236]:
# options、value、children？？？
from base64 import decode


app = JupyterDash('Chained Callbacks')
all_options = model

app.layout = html.Div([
    html.Div(
        [html.H4('售后服务数据')]
    ),
    html.Div(
        [html.P("产品类型:"),
        dcc.Dropdown(id = 'types',
                    options = [{'label': k, 'value': k} for k in all_options.keys()],
                    value = '成品', #初始值为成品
                    clearable = False
        ),
        html.P("规格型号:"),
        dcc.Dropdown(id = 'specifications',
                    clearable = False
        ),
        html.P("列名:"),
        dcc.Dropdown(id = 'columns',
                    options = ['问题现象', '问题原因', '解决方案'],
                    value = '问题现象', #初始值为问题现象
                    clearable = False
        )],
        style = dict(width = '30%', display = 'inline-block', float = 'right', padding = '80px 0px 0px 0px') #设置下拉列表的大小
                                                                                                                # padding是上左下右
    ),
    html.Div([
            dcc.Graph(id = "graph"),
        ],
        style = dict(width = '70%', display = 'inline-block',)
    ),
    html.Div([
            dcc.Graph(id = "该现象的问题原因"), 
            html.Div(id = "output")
        ],
        style = dict(width = '50%', display = 'inline-block',)
    ), 
    html.Div([
            dcc.Graph(id = "该现象的解决方案"),
        ],
        style = dict(width = '50%', display = 'inline-block', float='right')
    ),    
])

# 选择产品类型
@app.callback(
    Output('specifications', 'options'), #options即下面的return是一个列表嵌套一个字典
    [Input('types', 'value')])
def set_type_options(select_type): #select_type是传入的参数，表示选择成品还是组件
    return [{'label': i, 'value': i} for i in all_options[select_type]]

# 选择规格型号
@app.callback(
    Output('specifications', 'value'),
    [Input('specifications', 'options')])
def set_specification_value(available_options): #available_options表示上面返回的列表嵌套字典
    return available_options[0]['value'] #取列表的第一项，找到字典键为value对应的值，这里选择键label也一样的

# 根据问题现象来画图
@app.callback(
    Output("graph", "figure"), #需要更新的输出，即图像
    [Input("types", "value"), #需要更新的输入，有三个：产品类型、规格型号、列名
    Input("specifications", "value"),
    Input("columns", "value"),])
def generate_chart(types, specifications, columns):
    texts = []
    values = []
    i = len(texts)
    temp = data[data['规格型号']==specifications][columns].value_counts()

    texts.append(list(temp.index))
    values.append(list(temp))

    # 是否忽略缺失值
    def check_exist(textdata):
        ind = []
        check_str = ['mask1', 'mask2', 'mask3', '检测无问题']
        for str in check_str:
            if str in textdata:
                ind.append(textdata.index(str))
        return ind
    ind = sorted(check_exist(texts[i]), reverse=True)
    for j in ind:
        texts[i].pop(j)
        values[i].pop(j)

    # split_multiple(texts[i], values[i])
    # texts[i], values[i] = sum_multiple(texts[i], values[i])

    labels, numsdata = texts[i], values[i]
    df = pd.DataFrame(zip(labels, numsdata), columns=[columns, '次数'])
    

    ind = df.sort_values(by='次数', inplace=False, ascending=False)['次数'].index.tolist()
    # print(ind)
    colors = ['lightslategray',] * len(df)
    if len(ind) >= 10:
        length = 3
    else:
        length = 1
    for i in range(length):
        colors[ind[i]] = 'crimson'
    fig = go.Figure(
        data=[
            go.Bar(
                x = df['次数'],
                y = df[columns],
                text = df['次数'],
                marker_color = colors,
                orientation = 'h', #横向的条形图
            )
        ]
    )
    fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                        title_text='型号 '+ specifications +' 的主要'+ columns,)
    return fig

# 可视化某一问题现象对应的问题原因
@app.callback(
    Output("该现象的问题原因", "figure"), #需要更新的输出，即图像
    # Output("output", "children"), #可以查看输出结果
    [Input("graph", "hoverData"),
    Input("specifications", "value"),
    Input("columns", "value")])
def synchronization_1(hoverdata, specifications, columns):
    a = str(hoverdata)
    a = a[60:].split(",")[2].split(":")[1].split("'")[1]
    if columns == '问题现象':
        # return a[60:].split(",")[2].split(":")[1].split("'")[1]
        temp = data[data['规格型号']==specifications]
        temp = temp[temp['问题现象']==a]['问题原因'].value_counts()
        texts = list(temp.index)
        values = list(temp)
        dff = pd.DataFrame(zip(texts, values), columns=['问题原因', '次数'])
        fig = go.Figure(
            data=[
                go.Bar(
                    x = dff['次数'],
                    y = dff['问题原因'],
                    text = dff['次数'],
                    orientation = 'h', #横向的条形图
                )
            ]
        )
        fig.update_traces(texttemplate='%{text:.s}', textposition='outside')
        fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                            title_text='型号 '+ specifications +' 的主要问题原因')
        return fig

# 可视化某一问题现象对应的解决方案
@app.callback(
    Output("该现象的解决方案", "figure"), #需要更新的输出，即图像
    [Input("该现象的问题原因", "hoverData"),
    Input("specifications", "value"),
    Input("columns", "value")])
def synchronization_1(hoverdata, specifications, columns):
    a = str(hoverdata)
    a = a[60:].split(",")[2].split(":")[1].split("'")[1]
    if columns == '问题现象':
        # return a[60:].split(",")[2].split(":")[1].split("'")[1]
        temp = data[data['规格型号']==specifications]
        temp = temp[temp['问题原因']==a]['解决方案'].value_counts()
        texts = list(temp.index)
        values = list(temp)
        dff = pd.DataFrame(zip(texts, values), columns=['解决方案', '次数'])
        fig = go.Figure(
            data=[
                go.Scatter(
                    x = dff['解决方案'],
                    y = dff['次数'],
                    text = dff['次数'],
                    mode='lines+markers',
                    name='lines+markers',
                )
            ]
        )
        fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                            title_text='型号 '+ specifications +' 的主要解决方案')
        return fig


if __name__ == '__main__':
    # app.run_server(mode='inline', width='80%', height=800)
    app.run_server(mode='external', host='127.0.0.1', port=8060, debug=True)

Dash app running on http://127.0.0.1:8060/
